In [7]:
import os
import numpy as np
import pandas as pd
import nibabel as nib
import glob
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.image as mpimg
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,recall_score

In [2]:
classes = ['Alzheimer', 'MCI' , 'Normal']

data = {}
labels = {}
for class_ in classes:
    dirs_ = os.listdir(class_)
    data_ = []
    for dir_ in dirs_:
        files_path =os.path.join(os.path.join(class_, dir_), '*.nii')
        for file_ in glob.glob(files_path):
            temp_file = nib.load(file_).get_data()
            if temp_file.shape[0] != 64 or temp_file.shape[1] != 64 or temp_file.shape[2] !=48:
                continue
            data_.append(temp_file)
    data[class_] = np.array(data_)

working on Alzheimer...
working on MCI...
working on Normal...


In [3]:
temp_concat = np.concatenate((data['Alzheimer'][0], data['Alzheimer'][1]), axis=3)
for i in range(2, data['Alzheimer'].shape[0]):
    temp_concat = np.concatenate((temp_concat, data['Alzheimer'][i]), axis=3)
data['Alzheimer'] = temp_concat
print data['Alzheimer'].shape

(64, 64, 48, 6020)


In [4]:
# temp_concat = np.concatenate((data['Alzheimer _ 70'][0], data['Alzheimer _ 70'][1]), axis=3)
# for i in range(2, data['Alzheimer _ 70'].shape[0]):
#     temp_concat = np.concatenate((temp_concat, data['Alzheimer _ 70'][i]), axis=3)
# data['Alzheimer _ 70'] = temp_concat
# print data['Alzheimer _ 70'].shape

In [5]:
temp_concat = np.concatenate((data['MCI'][0], data['MCI'][1]), axis=3)
for i in range(2, data['MCI'].shape[0]):
    temp_concat = np.concatenate((temp_concat, data['MCI'][i]), axis=3)
data['MCI'] = temp_concat
print data['MCI'].shape

(64, 64, 48, 5320)


In [6]:
# temp_concat = np.concatenate((data['MCI _ 70'][0], data['MCI _ 70'][1]), axis=3)
# for i in range(2, data['MCI _ 70'].shape[0]):
#     temp_concat = np.concatenate((temp_concat, data['MCI _ 70'][i]), axis=3)
# data['MCI _ 70'] = temp_concat
# print data['MCI _ 70'].shape

In [7]:
temp_concat = np.concatenate((data['Normal'][0], data['Normal'][1]), axis=3)
for i in range(2, data['Normal'].shape[0]):
    temp_concat = np.concatenate((temp_concat, data['Normal'][i]), axis=3)
data['Normal'] = temp_concat
print data['Normal'].shape

(64, 64, 48, 5880)


In [8]:
# temp_concat = np.concatenate((data['Normal _ 70'][0], data['Normal _ 70'][1]), axis=3)
# for i in range(2, data['Normal _ 70'].shape[0]):
#     temp_concat = np.concatenate((temp_concat, data['Normal _ 70'][i]), axis=3)
# data['Normal _ 70'] = temp_concat
# print data['Normal _ 70'].shape

In [9]:
np.save('Normal',data['Normal'])

In [10]:
np.save('Alzheimer',data['Alzheimer'])

In [11]:
np.save('MCI',data['MCI'])

## Checkpoint


In [ ]:
normal=np.load('/home/siddhant/SharedFolder/DATA/Normal.npy')

In [ ]:
mci=np.load('/home/siddhant/SharedFolder/DATA/MCI.npy')

In [ ]:
alzheimer=np.load('/home/siddhant/SharedFolder/DATA/Alzheimer.npy')

In [ ]:
alzheimer.shape

In [ ]:
mci.shape

In [ ]:
normal.shape

In [ ]:
alzheimer=alzheimer.reshape(64,64,-1)
normal=normal.reshape(64,64,-1)
mci=mci.reshape(64,64,-1)

In [ ]:
combined_data=[]
for i in range(alzheimer.shape[2]):
    combined_data.append((alzheimer[:,:,i],0))
for i in range(mci.shape[2]):
    combined_data.append((mci[:,:,i],1))
for i in range(normal.shape[2]):
    combined_data.append((normal[:,:,i],2))
    
del(alzheimer)
del(normal)
del(mci)

In [ ]:
np.array(combined_data).shape

In [ ]:
# shuffling of combined_data
random.shuffle(combined_data)
print len(combined_data)

In [ ]:
# get input and output.
X = map(lambda x: x[0],combined_data)
y = map(lambda x:x[1], combined_data)
print len(y)

## Neural Network

###  Prepare Data

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(X,y, test_size = 0.2)

In [ ]:
np.array(y_val).shape

In [ ]:
def i16_to_rgb(array):
    img=Image.fromarray(np.asarray(array),'I;16')
    im2=img.convert('RGB')
    rgb_array=np.asarray(im2)
    return rgb_array

In [ ]:
x_val2=map(i16_to_rgb,x_val)

In [ ]:
x_train2=map(i16_to_rgb,x_train)

In [ ]:
del(x_train)
del(y_train)

In [4]:
short_x_train=x_train2[:10000]
short_y_train=y_train[:10000]

NameError: name 'x_train2' is not defined

In [5]:
short_x_val=x_val2[:1000]
short_y_val=y_val[:1000]

NameError: name 'x_val2' is not defined

In [1]:
# import the necessary packages
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D,Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras import optimizers
sgd = optimizers.SGD(lr=1, decay=1e-6, momentum=0.9, nesterov=True)

# initialize the model
model = Sequential()

#first layer
model.add(Conv2D(48, 5, 5, border_mode="same",
    input_shape=(64, 64,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

#set of FC
model.add(Flatten())

# softmax classifier
model.add(Dense(3))
model.add(Activation("softmax"))

#get model summary
model.summary()

# Compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 48)        3648      
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 48)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 48)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 49152)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 147459    
_________________________________________________________________
activation_2 (Activation)    (None, 3)                 0         
Total params: 151,107.0
Trainable params: 151,107.0
Non-trainable params: 0.0
________________________________________________________________

/home/siddhant/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), padding="same", input_shape=(64, 64, 3...)`


In [2]:
def batch_generator(X, y, batch_size,samples_per_epoch):
    number_of_batches = (samples_per_epoch)/batch_size
    counter=0
    while 1:
        X_batch = X[counter*batch_size:(counter+1)*batch_size]
        y_batch = y[counter*batch_size:(counter+1)*batch_size]
        counter += 1
        yield(np.array(X_batch),y_batch)
        if (counter == (number_of_batches - 1)):
            break

In [3]:
model.fit_generator(generator=batch_generator(short_x_train, short_y_train, 512,19999),
                    nb_epoch=25, samples_per_epoch=19999,
                    validation_data=batch_generator(short_x_val, short_y_val, 512,1999), nb_val_samples=1999)

NameError: name 'short_x_train' is not defined

In [51]:
model.fit(np.asarray(x_train2), np.asarray(y_train),nb_epoch=2,batch_size=256,
          validation_data=(np.asarray(x_val2),np.asarray(y_val)))

MemoryError: 

In [36]:
model.save_weights('Le-Net_model_weights') 

In [41]:
print(model.lr.get_value())

AttributeError: 'Sequential' object has no attribute 'lr'